In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils

In [3]:
#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

In [4]:
# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

In [6]:
os.path.join("Sign_Images")

'Sign_Images'

In [7]:
#path for flatten datas
dataPath = os.path.join('Sign_Data')

#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Hello","Food","Washroom"])

#30 video of each sign
numSequences=90

#length of each video
sequenceLength=30

In [8]:
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass
# if not os.path.exists("Sign_Data"):
#     os.mkdir("Sign_Data")
# else:
#     print("already exist")


In [8]:
os.path??

In [9]:
# while True:
#     inpt=input(print("Select which data are you collecting:\n0: Hello \n1: Thank you"))
#     if inpt == '0' or inpt == '1':
#         choice=signs[int(inpt)]
#         break
#     else:
#         print("Invalid input, please select again!!")
# choice   

In [9]:
vid=cv2.VideoCapture(0)
with mpHolistic.Holistic() as holistic:

    #taking label input 
     while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
            
    #checking camera is opened or not and taking data    
     while vid.isOpened() and inpt!=27 :
        for sequence in range(60,90):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'press any key to conitnue', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                   # key= cv2.waitKey(0)
                    cv2.waitKey(2000)
                    
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data
                
                if key == 27 : #press esc to close the window
                    break     

            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            

# Preprocess Data and Create Labels and Features

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
label_map = {label:num for num, label in enumerate(signs)}

In [8]:
label_map

{'Hello': 0, 'Food': 1, 'Washroom': 2}

In [9]:
sequences, labels = [], []
for sign in signs:
    for sequence in range(numSequences):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, sign, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [11]:
np.array(labels).shape

(90,)

In [12]:
np.array(sequences).shape

(90, 30, 1662)

In [13]:
arr=np.array([[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]]])

In [14]:
arr.shape

(2, 3, 2)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(90, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
y.shape

(90, 3)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [21]:
y_test.shape

(9, 3)

# Save test datas and signs

In [35]:
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
try:
    os.makedirs(os.path.join(testDatasPath))
    os.makedirs(os.path.join(signsPath))
except:
    pass
np.save(os.path.join(testDatasPath,'x_test.npy'),X_test)
np.save(os.path.join(testDatasPath,'y_test.npy'),y_test)
np.save(os.path.join(signsPath,'signs.npy'),signs)

# Build and Train LSTM Neural Network

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
def createModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(signs.shape[0], activation='softmax'))
    return model

In [25]:
model=createModel()

In [26]:
res = [.7, 0.2, 0.1]

In [27]:
signs[np.argmax(res)]

'Hello'

In [28]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [31]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 69ms/step - loss: 1.4000 - categorical_accuracy: 0.2346
Epoch 2/2000
3/3 [==============================] - 0s 68ms/step - loss: 5.6561 - categorical_accuracy: 0.3333
Epoch 3/2000
3/3 [==============================] - 0s 69ms/step - loss: 6.1024 - categorical_accuracy: 0.2593
Epoch 4/2000
3/3 [==============================] - 0s 71ms/step - loss: 7.2468 - categorical_accuracy: 0.3951
Epoch 5/2000
3/3 [==============================] - 0s 68ms/step - loss: 18.6775 - categorical_accuracy: 0.1481
Epoch 6/2000
3/3 [==============================] - 0s 80ms/step - loss: 26.4314 - categorical_accuracy: 0.3333
Epoch 7/2000
3/3 [==============================] - 0s 79ms/step - loss: 26.6299 - categorical_accuracy: 0.4074
Epoch 8/2000
3/3 [==============================] - 0s 65ms/step - loss: 21.5939 - categorical_accuracy: 0.1975
Epoch 9/2000
3/3 [==============================] - 0s 64ms/step - loss: 14.6802 - categorical_accuracy: 0.3

3/3 [==============================] - 0s 97ms/step - loss: 0.3748 - categorical_accuracy: 0.8148
Epoch 146/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.3570 - categorical_accuracy: 0.8395
Epoch 147/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.3701 - categorical_accuracy: 0.8395
Epoch 148/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.3291 - categorical_accuracy: 0.8519
Epoch 149/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.3653 - categorical_accuracy: 0.8272
Epoch 150/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.3695 - categorical_accuracy: 0.8148
Epoch 151/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.3075 - categorical_accuracy: 0.8395
Epoch 152/2000
3/3 [==============================] - 0s 96ms/step - loss: 0.2907 - categorical_accuracy: 0.8765
Epoch 153/2000
3/3 [==============================] - 0s 102ms/step - loss: 0.2906 - categorical_accuracy: 0.

3/3 [==============================] - 0s 96ms/step - loss: 0.5469 - categorical_accuracy: 0.8148
Epoch 218/2000
3/3 [==============================] - 0s 101ms/step - loss: 0.5000 - categorical_accuracy: 0.8148
Epoch 219/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.4543 - categorical_accuracy: 0.7901
Epoch 220/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.3545 - categorical_accuracy: 0.8395
Epoch 221/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.4375 - categorical_accuracy: 0.8148
Epoch 222/2000
3/3 [==============================] - 0s 132ms/step - loss: 0.3974 - categorical_accuracy: 0.8025
Epoch 223/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.3384 - categorical_accuracy: 0.8519
Epoch 224/2000
3/3 [==============================] - 0s 127ms/step - loss: 0.3364 - categorical_accuracy: 0.8519
Epoch 225/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.4204 - categorical_accuracy: 0

3/3 [==============================] - 0s 78ms/step - loss: 0.1897 - categorical_accuracy: 0.9259
Epoch 290/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.2081 - categorical_accuracy: 0.8889
Epoch 291/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.2514 - categorical_accuracy: 0.9012
Epoch 292/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.2054 - categorical_accuracy: 0.9259
Epoch 293/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1601 - categorical_accuracy: 0.9506
Epoch 294/2000
3/3 [==============================] - 0s 109ms/step - loss: 0.1955 - categorical_accuracy: 0.9136
Epoch 295/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.1949 - categorical_accuracy: 0.9136
Epoch 296/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.2405 - categorical_accuracy: 0.9012
Epoch 297/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.1775 - categorical_accuracy: 0.9

3/3 [==============================] - 0s 91ms/step - loss: 0.1121 - categorical_accuracy: 0.9630
Epoch 433/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.1016 - categorical_accuracy: 0.9753
Epoch 434/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.1026 - categorical_accuracy: 0.9753
Epoch 435/2000
3/3 [==============================] - 0s 126ms/step - loss: 0.0987 - categorical_accuracy: 0.9753
Epoch 436/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.0980 - categorical_accuracy: 0.9753
Epoch 437/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.1000 - categorical_accuracy: 0.9753
Epoch 438/2000
3/3 [==============================] - 0s 106ms/step - loss: 0.0944 - categorical_accuracy: 0.9753
Epoch 439/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.0933 - categorical_accuracy: 0.9753
Epoch 440/2000
3/3 [==============================] - 0s 129ms/step - loss: 0.0929 - categorical_accuracy:

...

3/3 [==============================] - 0s 85ms/step - loss: 0.1207 - categorical_accuracy: 0.9630
Epoch 1009/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1171 - categorical_accuracy: 0.9630
Epoch 1010/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.1196 - categorical_accuracy: 0.9753
Epoch 1011/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1387 - categorical_accuracy: 0.9383
Epoch 1012/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1308 - categorical_accuracy: 0.9383
Epoch 1013/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.1757 - categorical_accuracy: 0.9506
Epoch 1014/2000
3/3 [==============================] - 0s 96ms/step - loss: 0.2168 - categorical_accuracy: 0.9136
Epoch 1015/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.2614 - categorical_accuracy: 0.8765
Epoch 1016/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.1695 - categorical_accurac

Epoch 1080/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0822 - categorical_accuracy: 0.9630
Epoch 1081/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.0755 - categorical_accuracy: 0.9753
Epoch 1082/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.0779 - categorical_accuracy: 0.9630
Epoch 1083/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.0885 - categorical_accuracy: 0.9630
Epoch 1084/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.1079 - categorical_accuracy: 0.9506
Epoch 1085/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.0806 - categorical_accuracy: 0.9630
Epoch 1086/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.1196 - categorical_accuracy: 0.9383
Epoch 1087/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.0738 - categorical_accuracy: 0.9753
Epoch 1088/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.0675 - 

3/3 [==============================] - 0s 96ms/step - loss: 0.0098 - categorical_accuracy: 1.0000
Epoch 1223/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.0099 - categorical_accuracy: 1.0000
Epoch 1224/2000
3/3 [==============================] - 0s 112ms/step - loss: 0.0084 - categorical_accuracy: 1.0000
Epoch 1225/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.0114 - categorical_accuracy: 1.0000
Epoch 1226/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.0084 - categorical_accuracy: 1.0000
Epoch 1227/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.0103 - categorical_accuracy: 1.0000
Epoch 1228/2000
3/3 [==============================] - 0s 97ms/step - loss: 0.0085 - categorical_accuracy: 1.0000
Epoch 1229/2000
3/3 [==============================] - 0s 97ms/step - loss: 0.0086 - categorical_accuracy: 1.0000
Epoch 1230/2000
3/3 [==============================] - 0s 97ms/step - loss: 0.0085 - categorical_accur

3/3 [==============================] - 0s 135ms/step - loss: 0.0599 - categorical_accuracy: 0.9753
Epoch 1365/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.0550 - categorical_accuracy: 0.9630
Epoch 1366/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.0540 - categorical_accuracy: 0.9753
Epoch 1367/2000
3/3 [==============================] - 0s 119ms/step - loss: 0.0412 - categorical_accuracy: 1.0000
Epoch 1368/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.0473 - categorical_accuracy: 0.9877
Epoch 1369/2000
3/3 [==============================] - 0s 125ms/step - loss: 0.0490 - categorical_accuracy: 0.9877
Epoch 1370/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.0630 - categorical_accuracy: 0.9753
Epoch 1371/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.0387 - categorical_accuracy: 1.0000
Epoch 1372/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.0918 - categorical_a

KeyboardInterrupt: 

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make Prediction

In [36]:
#load signs, x_test, y_test
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))

In [37]:
res = model.predict(x_test)

In [38]:
signs

array(['Hello', 'Food', 'Washroom'], dtype='<U8')

In [39]:
signs[np.argmax(res[5])]

'Hello'

In [40]:
signs[np.argmax(y_test[5])]

'Hello'

In [40]:
res.shape

(9, 3)

In [41]:
y_test

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [42]:
res

array([[5.3766727e-02, 2.4770682e-01, 6.9852644e-01],
       [8.0655706e-01, 2.0600162e-02, 1.7284285e-01],
       [8.0121195e-01, 2.1262284e-02, 1.7752577e-01],
       [1.4012504e-06, 8.5017389e-01, 1.4982472e-01],
       [1.4011457e-06, 8.4682757e-01, 1.5317097e-01],
       [1.7091527e-06, 8.4000665e-01, 1.5999164e-01],
       [2.0998116e-06, 8.3362406e-01, 1.6637386e-01],
       [5.3696368e-07, 8.7554127e-01, 1.2445823e-01],
       [1.0858611e-06, 8.5759461e-01, 1.4240421e-01]], dtype=float32)

In [43]:
np.argmax(res[0])

2

In [44]:
np.argmax(y_test[0])

2

# Save Model

In [41]:
model.save('SLR.h5')

# Load Model

In [65]:
model=createModel()

In [66]:
model.load_weights('SLR.h5')

# Evaluation using confusion matrix

In [42]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix, accuracy_score

In [43]:
true_labels=[]
for label in y_test:
    index=np.argmax(label)
    true_labels.append(signs[index])

In [44]:
true_labels

['Washroom',
 'Washroom',
 'Washroom',
 'Hello',
 'Food',
 'Hello',
 'Food',
 'Food',
 'Hello']

In [45]:
y_predicted = model.predict(x_test)

In [46]:
predicted_labels=[]
for label in y_predicted:
    index=np.argmax(label)
    predicted_labels.append(signs[index])

In [47]:
predicted_labels

['Washroom',
 'Washroom',
 'Washroom',
 'Hello',
 'Food',
 'Hello',
 'Food',
 'Food',
 'Hello']

In [48]:
y_true = np.argmax(y_test, axis=1).tolist()

In [49]:
y_predicted=np.argmax(y_predicted,axis=1).tolist()

In [50]:
y_true

[2, 2, 2, 0, 1, 0, 1, 1, 0]

In [51]:
y_test

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [52]:
y_test.shape

(9, 3)

In [53]:
y_predicted

[2, 2, 2, 0, 1, 0, 1, 1, 0]

In [54]:
signs

array(['Hello', 'Food', 'Washroom'], dtype='<U8')

In [55]:
multilabel_confusion_matrix(true_labels, predicted_labels,labels=signs)

array([[[6, 0],
        [0, 3]],

       [[6, 0],
        [0, 3]],

       [[6, 0],
        [0, 3]]], dtype=int64)

In [56]:
confusion_matrix(y_true, y_predicted)

array([[3, 0, 0],
       [0, 3, 0],
       [0, 0, 3]], dtype=int64)

In [57]:
accuracy_score(y_true,y_predicted)

1.0

In [85]:
y_test.tolist()

[[0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0]]

In [100]:
list(zip(true_labels, predicted_labels))

[('Food', 'Food'),
 ('Food', 'Food'),
 ('Hungry', 'Hungry'),
 ('Thank You', 'Thank You'),
 ('Thank You', 'Thank You'),
 ('Thank You', 'Thank You'),
 ('Hello', 'Hello'),
 ('Food', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Hungry', 'Hungry'),
 ('Food', 'Food')]

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils
vid=cv2.VideoCapture(0)

#path for flatten datas
dataPath = os.path.join('Sign_Data')

key=1


#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Hello","Thank You", "Hungry", "Food", "Hospital", "Washroom"])

#30 video of each sign
numSequences=60


#length of each video
sequenceLength=30

#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

# making folders if not exist
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass

with mpHolistic.Holistic() as holistic:

    #taking label input 
    while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
                     
    #checking camera is opened or not and taking data    
    while vid.isOpened() and inpt!=27 :
        for sequence in range(30,60):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'starts in 2 sec', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                    key= cv2.waitKey(2000)
                    # cv2.waitKey(2000)
                
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data


                if key == 27 : #press esc to close the window
                    break     
            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            